In [ ]:
import torch
import matplotlib.pyplot as plt
from torchvision.datasets import EMNIST
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
from collections import Counter
import torchvision.transforms.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [ ]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: F.rotate(x, -90)),  
    transforms.Lambda(lambda x: F.hflip(x)),       
    transforms.Normalize((0.5,), (0.5,))
])

emnist_train = EMNIST(root='./data', split='balanced', train=True, download=False, transform=transform)

emnist_test = EMNIST(root='./data', split='balanced', train=False, download=False, transform=transform)
emnist_mapping = emnist_train.classes

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def show_samples(dataset, mapping, n=20):
    plt.figure(figsize=(15, 3))
    for i in range(n):
        image, label = dataset[i]
        image = image.squeeze().numpy()
        char = mapping[label]        
        plt.subplot(2, n//2, i+1)
        plt.imshow(image, cmap='gray')
        plt.title(f"Label: {label}\nChar: {char}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()
show_samples(emnist_train, emnist_mapping)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(42)
np.random.seed(42)


# Model Architecture 
class EMNIST_CNN(nn.Module):
    def __init__(self, num_classes=47):
        super().__init__()
        self.conv_layers = nn.Sequential(
            # Conv Block 1
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Conv Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Conv Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 3 * 3, 256),  
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  
        x = self.fc_layers(x)
        return x

In [ ]:
batch_size = 64
learning_rate = 0.001
num_epochs = 20

In [ ]:
train_loader = DataLoader(emnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(emnist_test, batch_size=batch_size, shuffle=False)

In [ ]:
model = EMNIST_CNN(num_classes=len(emnist_mapping)).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

In [ ]:
def train_model():
    train_loss_history = []
    val_loss_history = []
    accuracy_history = []
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        
        epoch_loss = running_loss / len(train_loader)
        train_loss_history.append(epoch_loss)
        
        
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_loss = val_loss / len(test_loader)
        val_loss_history.append(val_loss)
        accuracy = 100 * correct / total
        accuracy_history.append(accuracy)
        
        scheduler.step(val_loss)
        
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {epoch_loss:.4f}, '
              f'Val Loss: {val_loss:.4f}, '
              f'Accuracy: {accuracy:.2f}%')
    
    return train_loss_history, val_loss_history, accuracy_history


In [ ]:
train_loss, val_loss, accuracy = train_model()

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(accuracy, label='Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Save both the model and class mapping
torch.save({
    'model_state_dict': model.state_dict(),
    'class_mapping': emnist_mapping,
    'input_size': (1, 28, 28)  
}, 'cnn_model.pth')